# Тестовое задание на позицию продуктового аналитика

[Ссылка на вакансию](https://hh.ru/vacancy/115519135)<br>
Выполнил: Дмитрий Поликарпов<br>
troyan27@yandex.ru<br>
tg: @vendor62x<br><br>


## Задача
У нас есть 2 таблицы с данными описанные ниже. Они покрывают события типа:
- Поисковые события
- Клики на сайтах
	- Клик на товар (eventType - product_click)
	- Добавление в корзину (eventType - card_add_event)
	- Клик на коризну (eventType - card_click_event)
	- Клик на завершение оплаты (eventType - order_click)

Они хранятся в базе данных Clickhouse (но можно использовать любой sql диалект)

Задача: Вам необходимо предложить метрики, методы их расчета и визуализации на основе имеющихся данных. Доступные инструменты для использования:
SQL
Python
BI системы (можно описать график который использовали бы для визуализации, пример - линейный график с осями.....)
Кол-во метрик и глубину погружения определяете сами.


Таблица `Searches`

| Column Name       | Description                                                                 |
|-------------------|-----------------------------------------------------------------------------|
| timestamp         | Дата и время события                                                         |
| remoteHost        | IP-адрес или хост пользователя, с которого поступил запрос.                 |
| location          | Текущий URL                                                                  |
| referer           | URL-источник, откуда пользователь перешёл                                    |
| regionId          | Идентификатор региона пользователя                                           |
| channel           | Тип браузера                                                                |
| eventType         | Тип события                                                                  |
| sessionId         | Уникальный идентификатор сессии                                              |
| userId            | Уникальный идентификатор пользователя                                        |
| userGUID          | GUID пользователя (генерируется при установке приложения)                    |
| viewGUID          | Уникальный GUID для конкретного просмотра/экрана/страницы                   |
| pageNumber        | Номер страницы (например, для пагинации результатов поиска)                 |
| searchTerm        | Поисковый запрос, введённый пользователем                                     |
| pageProducts      | Список товаров, отображённых на текущей странице                             |
| searchWorked      | Признак, указывающий, сработал ли поисковый движок                           |
| synonymsWorked    | Признак, указывающий, сработал ли поисковый движок с использованием синонимов|
| originalSearchTerm| Исходный поисковый запрос пользователя (до применения автокоррекции и т. д.) |
| isFromRedirect    | Признак, указывающий, был ли редирект на конкретную страницу после поиска     |
| isZeroQuery       | Признак «нулевого» запроса, когда пользователь не вводил ключевых слов.      |


Таблица `Widget clicks`

| Column Name       | Description                                                                 |
|-------------------|-----------------------------------------------------------------------------|
| timestamp         | Временная метка события (DateTime в часовом поясе Europe/Moscow).            |
| remoteHost        | IP-адрес или доменное имя клиента, с которого поступил запрос.             |
| location          | Текущий URL                                                                  |
| referer           | URL-источник, откуда пользователь перешёл                                    |
| regionId          | Идентификатор региона пользователя (или сессии)                              |
| channel           | Тип браузера                                                                |
| eventType         | Тип события (клик, просмотр, покупка, переход и т. д.)                       |
| sessionId         | Уникальный идентификатор сессии                                              |
| userGUID          | GUID пользователя (генерируется при установке приложения или при первой сессии на сайте) |
| viewGUID          | Уникальный GUID для конкретного просмотра/экрана/страницы                   |
| pageNumber        | Номер страницы (например, при пагинации результатов поиска или списка товаров) |
| productId         | Идентификатор конкретного продукта (если событие связано с товаром)          |
| position          | Позиция товара или элемента на странице (например, в списке результатов)    |
| widgetType        | Тип виджета/блока, в котором отображается контент (например, рекомендации, баннер) |


## Решение

### Воронка конверсии

#### 1. CTR поисковых запросов (Search to Product Click) <br>

Допустим, что все записи в таблице `Searches` относятся к поисковым запросам, тогда нужное разделение на поисковые события и клики на товары мы делаем по значениям столбца `eventType` в таблице `Widget clicks`. Для вычисления метрики нужно посчитать, сколько пользователей, сделавших поисковой запрос, кликнули на товар на страницах с результатами поиска.

```sql
WITH search_data AS (
    SELECT
        DISTINCT sessionId,
        toDate(timestamp) AS event_date
    FROM Searches
),

product_click_data AS (
    SELECT
        DISTINCT sessionId,
        toDate(timestamp) AS event_date
    FROM `Widget clicks`
    WHERE eventType = 'product_click'
)

SELECT
    s.event_date,
    COUNT(DISTINCT p.sessionId) / COUNT(DISTINCT s.sessionId) * 100 AS search_conversion_rate
FROM search_data AS s
LEFT JOIN product_click_data AS p ON s.sessionId = p.sessionId AND s.event_date = p.event_date
GROUP BY s.event_date
ORDER BY s.event_date;
```


- Считаем по `sessionId`, потому что нужно оценить, сколько уникальных сессий с поисковыми запросами завершились кликом на товар.
- Отслеживаем динамику по дням.
- Визуализация:
  - линейный график, показывающий изменение конверсии поиска во времени. Временная ось на X и значение CTR на Y.
  - аналогичная столбчатая диаграмма.
- аналогичным образом можно посчитать отношение добавлений в корзину, кликов на корзину или завершений оплаты к количеству поисковых запросов.

#### 2. CTR добавлений в корзину (Click to Add to Cart).<br>

Считаем сколько сессий, в которых был выполнен клик по товару, завершились добавлением товара в корзину.

```sql
WITH product_click_data AS (
    SELECT DISTINCT sessionId,
           toDate(timestamp) AS event_date
    FROM `Widget clicks`
    WHERE eventType = 'product_click'
),

cart_add_data AS (
    SELECT DISTINCT sessionId,
           toDate(timestamp) AS event_date
    FROM `Widget clicks`
    WHERE eventType = 'card_add_event'
)

SELECT
    p.event_date,
    COUNT(DISTINCT c.sessionId) / COUNT(DISTINCT p.sessionId) * 100 AS click_to_cart_ctr
FROM product_click_data AS p
LEFT JOIN cart_add_data AS c ON p.sessionId = c.sessionId AND p.event_date = c.event_date
GROUP BY p.event_date
ORDER BY p.event_date;

```

- Визуализация: линейная диаграмма или столбчатый график.

#### 3. Расчет кликов по корзине и завершения оплаты.
Аналогичным образом считаем конверсию переходов к оплате (card_click_event / card_add_event) и завершений заказа (order_click / card_click_event). Таким образом мы сможем визуализировать и отслеживать динамику на каждом из четырёх этапов.

#### 4. Построение воронки конверсии.

Объединяем все этапы в один запрос для формирования таблицы.

```sql
WITH search_data AS (
    SELECT DISTINCT sessionId,
           toDate(timestamp) AS event_date
    FROM Searches
),
product_click_data AS (
    SELECT DISTINCT sessionId,
           toDate(timestamp) AS event_date
    FROM `Widget clicks`
    WHERE eventType = 'product_click'
),
cart_add_data AS (
    SELECT DISTINCT sessionId,
           toDate(timestamp) AS event_date
    FROM `Widget clicks`
    WHERE eventType = 'card_add_event'
),
order_click_data AS (
    SELECT DISTINCT sessionId,
           toDate(timestamp) AS event_date
    FROM `Widget clicks`
    WHERE eventType = 'order_click'
)

SELECT
    s.event_date,
    COUNT(DISTINCT s.sessionId) AS search_count,
    COUNT(DISTINCT p.sessionId) AS product_click_count,
    COUNT(DISTINCT c.sessionId) AS cart_add_count,
    COUNT(DISTINCT o.sessionId) AS order_click_count
FROM search_data AS s
LEFT JOIN product_click_data AS p ON s.sessionId = p.sessionId
LEFT JOIN cart_add_data AS c ON s.sessionId = c.sessionId
LEFT JOIN order_click_data AS o ON s.sessionId = o.sessionId
GROUP BY s.event_date
ORDER BY s.event_date;
```

- Такую таблицу можно выгрузить в BI для создания отчета, который будет отображать метрики по каждому этапу воронки на каждой дате.
- В BI-системах можно добавить фильтрацию по дате, чтобы отслеживать воронку для конкретного дня или периода времени.
- Для визуализации воронки можно использовать графики типа диаграммы воронки или горизонтальной столбчатой диаграммы, чтобы продемонстрировать, сколько сессий прошло через каждый этап воронки.

### Поведение пользователей

#### 1. Рейтинг удержания.<br>

Используем колонку `userId` из таблицы `Searches`, чтобы определить первую активность пользователя. Затем проверяем, когда этот пользователь возвращался (через день, через неделю и т. д.). Retention Rate - отношение числа вернувшихся пользователей к общему количеству пользователей.

```sql
WITH first_interaction AS (
    -- Находим дату первого взаимодействия для каждого пользователя
    SELECT
        userId,
        MIN(DATE(timestamp)) AS first_date
    FROM Searches
    GROUP BY userId
),
daily_interactions AS (
    -- Определяем дни взаимодействий пользователей относительно их первого дня
    SELECT
        f.userId,
        f.first_date,
        DATE(s.timestamp) AS event_date,
        DATEDIFF(DATE(s.timestamp), f.first_date) AS days_since_first
    FROM first_interaction AS f
    JOIN Searches AS s
      ON f.userId = s.userId
    WHERE DATEDIFF(DATE(s.timestamp), f.first_date) >= 0
),
retention_summary AS (
    -- Считаем, сколько пользователей взаимодействовали в каждый день
    SELECT
        days_since_first,
        COUNT(DISTINCT userId) AS retained_users
    FROM daily_interactions
    GROUP BY days_since_first
),
total_users AS (
    -- Определяем общее количество пользователей, совершивших первое взаимодействие
    SELECT
        COUNT(DISTINCT userId) AS total_users
    FROM first_interaction
)
-- Рассчитываем Retention Rate
SELECT
    r.days_since_first,
    r.retained_users,
    t.total_users,
    (r.retained_users / t.total_users) * 100 AS retention_rate
FROM retention_summary AS r
CROSS JOIN total_users AS t
ORDER BY r.days_since_first;

```

- Такую таблицу можно визуализировать линейным графиком для отображения RR по дням.
- RR можно также рассчитывать по кликам, добавлениям в корзину и покупкам, используя stacked bar chart, чтобы показать сравнение.
- Можно применить методы когортного анализа, сгруппировав пользователей, например, по дате их привлечения и в дальнейшем отслеживать поведение когорты.
- RR по когортам можно визуализировать также линейным графиком или тепловой картой, используя при этом кумулятивный подсчёт, который более удобен для сравнения когорт.

#### 2. Показатель "покинутой корзины" по регионам.<br>

Эта метрика покажет, какой процент пользователей из разных регионов покидает корзину, не совершив покупку.

```sql
WITH add_to_cart AS (
    SELECT DISTINCT sessionId, regionId
    FROM `Widget clicks`
    WHERE eventType = 'card_add_event'
),
order_click AS (
    SELECT DISTINCT sessionId, regionId
    FROM `Widget clicks`
    WHERE eventType = 'order_click'
)

SELECT
    a.regionId,
    COUNT(DISTINCT a.sessionId) / COUNT(DISTINCT o.sessionId) * 100 AS cart_abandonment_rate
FROM add_to_cart As a
LEFT JOIN order_click AS o ON a.sessionId = o.sessionId
GROUP BY a.regionId;

```

- Вместо региона можно использовать тип браузера или id продукта, чтобы выявить возможные проблемы, препятствующие совершению покупки.

#### 3. Расчет времени от поиска до покупки.

```sql
WITH search_data AS (
    SELECT
        userId,
        sessionId,
        MIN(timestamp) AS search_time  -- Время первого поиска в сессии
    FROM
        Searches
    GROUP BY
        userId, sessionId
),

purchase_data AS (
    SELECT
        userId,
        sessionId,
        MIN(timestamp) AS purchase_time  -- Время первого клика на покупку в сессии
    FROM
        `Widget clicks`
    WHERE
        eventType = 'order_click'
    GROUP BY
        userId, sessionId
)

SELECT
    s.userId,
    s.sessionId,
    TIMESTAMPDIFF(SECOND, s.search_time, p.purchase_time) AS time_to_purchase_seconds
FROM
    search_data AS s
JOIN
    purchase_data AS p
    ON s.userId = p.userId AND s.sessionId = p.sessionId
WHERE
    TIMESTAMPDIFF(SECOND, s.search_time, p.purchase_time) > 0;

```


- Такую таблицу можно визуализировать несколькими способами:
  - Гистограмма покажет распределение времени, которое пользователи тратят от поиска до покупки. Можно увидеть, сколько пользователей совершили покупку в течение определённого промежутка времени.
  - Boxplot позволит увидеть медиану времени, а также выбросы и диапазоны для времени между поиском и покупкой.
  - Scatter plot поможет визуализировать индивидуальные данные о времени от поиска до покупки для каждого пользователя. Это может помочь выявить аномалии или необычные паттерны.

### Технические метрики

#### 1. Процент неудачных поисков.

```sql
SELECT
    DATE(timestamp) AS search_date,
    (COUNT(CASE
                WHEN searchWorked = 0
                AND isZeroQuery = 0
            THEN 1
            END) /
     COUNT(CASE
                WHEN isZeroQuery = 0
            THEN 1
            END) * 100) AS unsuccessful_search_rate
FROM Searches
GROUP BY DATE(timestamp)
ORDER BY search_date;

```

Подсчет и визуализация процента неудачных поисков в динамике помогут:
- Оценить эффективность работы поисковой системы с течением времени.
- Выявить и проанализировать аномалии в поисковых запросах.
- Понять, как изменения на сайте или в поисковом движке влияют на результаты поиска.

#### 2. Анализ неудачных запросов.

```sql
SELECT
    searchTerm,
    COUNT(*) AS unsuccessful_search_count
FROM Searches
WHERE searchWorked = 0
    AND isZeroQuery = 0
GROUP BY searchTerm
ORDER BY unsuccessful_search_count DESC;

```

Анализ такой таблицы поможет:
- Выявлять проблемы с поисковым движком.
- Находить недостаточно покрытые запросы.
- Прогнозировать тренды и потребности пользователей.

#### 3. Среднее время отклика поиска.

С помощью оконной функции вычислим среднее время между успешными поисковыми запросами в рамках одной сессии.

```sql
SELECT
    AVG(response_time) AS avg_search_response_time
FROM (
    SELECT
        EXTRACT(EPOCH FROM (
            timestamp - LAG(timestamp) OVER (
                PARTITION BY sessionId
                ORDER BY timestamp
            )
        )) AS response_time
    FROM
        Searches
    WHERE
        isZeroQuery = 0
        AND searchWorked = 1
) AS response_times
WHERE
    response_time IS NOT NULL;
```

Такой запрос может помочь:
- Обнаруживать технические проблемы с поисковым движком.
- Анализировать производительность интерфейса.
- Оценивать пользовательское поведение.